# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [26]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [45]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [50]:
le = LabelEncoder()
le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,1,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,1,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,3,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,1,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [51]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)

# Create our target
y = df["loan_status"]

In [47]:
X.describe(include='all')

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817,68817,68817,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817,68817
unique,NaN,NaN,NaN,NaN,NaN,3,3,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
top,NaN,NaN,NaN,NaN,NaN,Not Verified,Jan-2019,n,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N
freq,NaN,NaN,NaN,NaN,NaN,32895,31041,68817,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68817,68817
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,NaN,NaN,NaN,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,NaN,NaN
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,NaN,NaN,NaN,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,NaN,NaN
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,NaN,NaN,NaN,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,NaN,NaN
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,NaN,NaN,NaN,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,NaN,NaN
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,NaN,NaN,NaN,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,NaN,NaN
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,NaN,NaN,NaN,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,NaN,NaN


In [52]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [55]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape

(51612, 95)

In [54]:
X = pd.get_dummies(X, columns=["home_ownership", "verification_status", "issue_d", 
                               "pymnt_plan","initial_list_status", "next_pymnt_d", 
                               "application_type", "hardship_flag", "debt_settlement_flag"])

X.head(10)

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1
5,25000.0,0.1797,634.43,90000.0,18.49,1.0,1.0,14.0,0.0,23889.0,...,1,1,0,1,0,1,1,0,1,1
6,20400.0,0.2000,540.48,51315.0,25.63,0.0,2.0,8.0,1.0,2569.0,...,1,1,0,1,0,1,1,0,1,1
7,10000.0,0.1557,349.46,66000.0,9.00,0.0,0.0,11.0,0.0,9407.0,...,1,1,0,1,0,1,1,0,1,1
8,14000.0,0.0881,443.96,45000.0,10.24,0.0,1.0,9.0,0.0,7058.0,...,1,1,1,0,0,1,1,0,1,1
9,10000.0,0.1474,345.39,63295.0,39.34,0.0,0.0,27.0,0.0,25790.0,...,1,1,0,1,0,1,1,0,1,1


## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [56]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [62]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [63]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [64]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [65]:
# Calculated the balanced accuracy score
# Calculated the balanced accuracy score
y_pred_brf = brf.predict(X_test_scaled)
bas_brf=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_brf)

0.7841019755929702


In [66]:
# Display the confusion matrix
cm_brf = confusion_matrix(y_test, y_pred_brf)
cm_df_brf = pd.DataFrame(
    cm_brf, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_brf

,Predicted High Risk,Predicted Low Risk
Actual High Risk,69,35
Actual Low Risk,1629,15472


In [67]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.66      0.90      0.08      0.77      0.59       104
   low_risk       1.00      0.90      0.66      0.95      0.77      0.61     17101

avg / total       0.99      0.90      0.66      0.94      0.77      0.61     17205



In [74]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07296730174469096, 'total_rec_prncp'),
 (0.062048326652909117, 'total_rec_int'),
 (0.06171791787194709, 'total_pymnt'),
 (0.060371849047589465, 'last_pymnt_amnt'),
 (0.05794847512773343, 'total_pymnt_inv'),
 (0.030784943998314808, 'int_rate'),
 (0.01709870603805275, 'installment'),
 (0.01684892283742061, 'issue_d_Jan-2019'),
 (0.016825581122403725, 'revol_bal'),
 (0.01680753410145557, 'dti'),
 (0.016573401032559058, 'annual_inc'),
 (0.016448353203836533, 'max_bal_bc'),
 (0.015988183431980135, 'il_util'),
 (0.01550502971915058, 'mo_sin_old_rev_tl_op'),
 (0.015180153579293004, 'out_prncp'),
 (0.014985954246848654, 'tot_hi_cred_lim'),
 (0.014581428593477034, 'mo_sin_old_il_acct'),
 (0.01434996167804375, 'bc_util'),
 (0.014288371762322758, 'out_prncp_inv'),
 (0.013954096710145268, 'tot_cur_bal'),
 (0.013627143270730294, 'issue_d_Mar-2019'),
 (0.013585916099843885, 'total_rev_hi_lim'),
 (0.013337051922040008, 'total_bc_limit'),
 (0.013205580542892621, 'total_bal_ex_mort'),
 (0.013156939

### Easy Ensemble Classifier

In [69]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train_scaled, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [70]:
# Calculated the balanced accuracy score
# Calculated the balanced accuracy score
y_pred_eec = eec.predict(X_test_scaled)
bas_eec=balanced_accuracy_score(y_test, y_pred_brf)
print(bas_eec)

0.7841019755929702


In [71]:
# Display the confusion matrix
cm_eec = confusion_matrix(y_test, y_pred_eec)
cm_df_eec = pd.DataFrame(
    cm_eec, index=["Actual High Risk", "Actual Low Risk"], columns=["Predicted High Risk", "Predicted Low Risk"]
)
cm_df_eec

,Predicted High Risk,Predicted Low Risk
Actual High Risk,93,11
Actual Low Risk,973,16128


In [73]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.89      0.94      0.16      0.92      0.84       104
   low_risk       1.00      0.94      0.89      0.97      0.92      0.85     17101

avg / total       0.99      0.94      0.89      0.97      0.92      0.85     17205



In [35]:
# Fit the SMOTE model to the data and check the count of each class
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({0: 17532, 1: 17532})

In [35]:
# Fit the SMOTE model to the data and check the count of each class
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({0: 17532, 1: 17532})

In [35]:
# Fit the SMOTE model to the data and check the count of each class
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)
from collections import Counter

Counter(y_resampled)

Counter({0: 17532, 1: 17532})

### Final Questions

1. Which model had the best balanced accuracy score?
    Balanced Random Forest Classifier  
    

2. Which model had the best recall score?

    balanced ramndom forrest classifier has the best recall  scores 

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.

4. What are the top three features?
  Total Debt
  Borrower Income
  Debt to Income Ratio